# Deep Learning I

1. Gradient Deescend

2. ¿Cómo aprender de los datos?
    * Square / Euclidean Loss
    * Hinge / Margin Loss (es decir, Support Vector Machine)
    * Pérdida Logística (Regresión Logística)
    * Sigmoid Cross-Entropy Loss (clasificador Softmax)
    * Descenso de gradiente por lotes (Batch Gradient Descend)
    * Stochastic Gradient Descend
3. Mini-batch Gradient Descend (Descenso del gradiente por mini-lotes)

4. Diferenciación Automática

## Previa: Optimización.

Lo más sencillo que podríamos intentar para minimizar una función $f(x)$ sería muestrear dos puntos relativamente cercanos entre sí, y simplemente dar pasos repetidos alejándonos del valor más alto. Este simple algoritmo tiene una limitación severa: no puede acercarse más al mínimo verdadero que el tamaño del paso.

El método **Nelder-Mead** ajusta dinámicamente el tamaño del paso en función de la pérdida del nuevo punto. Si el nuevo punto es mejor que cualquier valor visto previamente, expande el tamaño del paso para acelerar hacia el fondo. De igual manera, si el nuevo punto es peor, contrae el tamaño del paso para converger alrededor del mínimo. Los ajustes usuales son reducir a la mitad el tamaño del paso al contraer y duplicar el tamaño del paso al expandir.

Este método se puede extender fácilmente a ejemplos de dimensiones superiores, todo lo que se necesita es tomar un punto más de los que hay dimensiones. Luego, el enfoque más simple es reemplazar el peor punto con un punto reflejado a través del centroide de los demás puntos n. Si este punto es mejor que el mejor punto actual, entonces podemos intentar estirar exponencialmente a lo largo de esta línea. Por otro lado, si este nuevo punto no es mucho mejor que el valor anterior, entonces estamos cruzando un valle, por lo que reducimos el paso hacia un punto mejor.

> Ver ["Un Tutorial Interactivo sobre Optimización Numérica"](http://www.benfrederickson.com/numerical-optimization/)

Preguntas:

+ ¿Cuáles son las limitaciones de este método desde un punto de vista computacional?
+ ¿En qué casos es una alternativa real?

Los métodos de optimización **basados en gradientes** proporcionan una alternativa a los enfoques de muestreo.


# 1. Descenso de Gradiente (Gradient Descend)

Supongamos que tenemos una función $f(w): \mathbf{R} \rightarrow \mathbf{R}$ y que nuestro objetivo es encontrar el argumento $w$ que minimiza esta función (para maximización, considera $-f(w)$). Para este fin, el concepto crítico es la *derivada*.

La derivada de $f$ de una variable $w$, $ f'(w)$ o $\frac{\delta f}{\delta w}$, es una medida de la tasa a la que el valor de la función cambia con respecto al cambio de la variable. Se define como el siguiente límite:

$$ f'(w) = \lim_{h \rightarrow 0} \frac{f(w + h) - f(w)}{h} $$

La derivada especifica cómo escalar un pequeño cambio en la entrada para obtener el cambio correspondiente en la salida. Conociendo el valor de $f(w)$ en un punto $w$, esto permite predecir el valor de la función en un punto vecino:

$$ f(w + h) \approx f(w) + h f'(w)$$

## 1.1. Primer enfoque

Entonces, siguiendo estos pasos podemos disminuir el valor de la función:

+ Comenzar desde un valor $w^0$ aleatorio.
+ Calcular la derivada $f'(w) = \lim_{h \rightarrow 0} \frac{f(w + h) - f(w)}{h}$.
+ Caminar pequeños pasos en la dirección opuesta de la derivada, $w^{i+1} = w^i - h f'(w^i)$, porque sabemos que $f(w - h f'(w))$ es menor que $f(w)$ para $h$ suficientemente pequeño, hasta que $ f'(w) \approx 0$.

La búsqueda del mínimo termina cuando la derivada es cero porque no tenemos más información sobre en qué dirección movernos. $w$ se llama un punto crítico o estacionario de $f(w)$ si $f'(w)=0$.

Todos los puntos extremos (máximos/mínimos) son puntos críticos porque $f(w)$ es menor/mayor que en todos los puntos vecinos. Pero estos no son los únicos puntos críticos: hay una tercera clase de puntos críticos llamados *puntos de muertos* (saddle points). Los puntos muertos son puntos que tienen derivadas parciales igual a cero pero en los cuales la función no tiene ni un valor máximo ni mínimo.

Si $f$ es una *función convexa*, cuando la derivada es cero, este debería ser el extremo de nuestra función. En otros casos podría ser un mínimo/máximo local o un punto muerto.


In [ ]:
# numerical derivative at a point x by using finite differences

def f(x):
    return x**2

def fin_dif(x, 
            f, 
            h = 0.00001):
    '''
    This method returns the derivative of f at x
    by using the finite difference method
    '''
    return (f(x+h) - f(x))/h

x = 2.0
print("{:2.4f}".format(fin_dif(x,f)))

> NOTA: Se puede demostrar que la "fórmula de diferencia centrada" es mejor al calcular derivadas numéricas:

> $$ \lim_{h \rightarrow 0} \frac{f(x + h) - f(x - h)}{2h} $$
El error en la aproximación de "diferencia finita" se puede derivar del teorema de Taylor y, asumiendo que $f$ es diferenciable, es $O(h)$. En el caso de la "diferencia centrada", el error es $O(h^2)$.

Hay dos problemas con las derivadas numéricas:

+ Es aproximado.
+ Es muy lento de evaluar (dos evaluaciones de función: $f(x + h), f(x - h)$).

¡Nuestros conocimientos de Cálculo podrían ayudar!

## 1.2. Segundo enfoque

Para encontrar el mínimo local usando gradient descend y en el caso de conocer una expresión analítica de la derivada de la **función** que queremos minimizar, puedes comenzar en un punto aleatorio y moverte en la dirección de descenso más pronunciado en relación con la derivada:

+ Comenzar desde un valor $x$ aleatorio.
+ Calcular la derivada $f'(x)$ analíticamente.
+ Dar un pequeño paso en la dirección opuesta de la derivada.

In [ ]:
#@title Default title text
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.datasets import make_regression 
from scipy import stats 
%matplotlib inline

x = np.linspace(-10,20,100)
y = x**2 - 6*x + 5
start = 15

fig, ax = plt.subplots(1, 1)
fig.set_facecolor('#EAEAF2')
plt.plot(x,y, 'r-')
plt.plot([start],[start**2 - 6*start + 5],'o')
ax.text(start,
        start**2 - 6*start + 35,
        'Start',
        ha='center',
        color=sns.xkcd_rgb['blue'],
       )

d = 2 * start - 6
end = start - d

plt.plot([end],[end**2 - 6*end + 5],'o')
plt.ylim([-10,250])
plt.gcf().set_size_inches((10,3))
plt.grid(True)
ax.text(end,
        start**2 - 6*start + 35,
        'End',
        ha='center',
        color=sns.xkcd_rgb['green'],
       )
plt.show

¡Hay un problema! ¿Cuál?

Necesitamos definir un *paso* adecuado para modular el valor del gradiente.

In [ ]:
old_min = 0
temp_min = 15
step_size = 0.01
precision = 0.0001

def f(x):
    return x**2 - 6*x + 5
    
def f_derivative(x):
    import math
    return 2*x -6

mins = []
cost = []

while abs(temp_min - old_min) > precision:
    old_min = temp_min 
    gradient = f_derivative(old_min) 
    move = gradient * step_size
    temp_min = old_min - move
    cost.append((3-temp_min)**2)
    mins.append(temp_min)

# rounding the result to 2 digits because of the step size
print("Local minimum occurs at {:3.6f}.".format(round(temp_min,2)))

In [ ]:
x = np.linspace(-10,20,100)
y = x**2 - 6*x + 5

x, y = (zip(*enumerate(cost)))

fig, ax = plt.subplots(1, 1)
fig.set_facecolor('#EAEAF2')
plt.plot(x,y, 'r-', alpha=0.7)
plt.ylim([-10,150])
plt.gcf().set_size_inches((10,3))
plt.grid(True)
plt.show

In [ ]:
x = np.linspace(-10,20,100)
y = x**2 - 6*x + 5

fig, ax = plt.subplots(1, 1)
fig.set_facecolor('#EAEAF2')
plt.plot(x,y, 'r-')
plt.ylim([-10,250])
plt.gcf().set_size_inches((10,3))
plt.grid(True)
plt.plot(mins,cost,'o', alpha=0.3)
ax.text(start,
        start**2 - 6*start + 25,
        'Start',
        ha='center',
        color=sns.xkcd_rgb['blue'],
       )
ax.text(mins[-1],
        cost[-1]+20,
        'End (%s steps)' % len(mins),
        ha='center',
        color=sns.xkcd_rgb['blue'],
       )
plt.show

#### Alpha

El tamaño del paso, **alfa**, es un concepto delicado: ya que si es demasiado pequeño convergeremos lentamente a la solución, pero si es demasiado grande podemos divergir de la solución. 

Hay varias políticas a seguir a la hora de seleccionar el tamaño del paso:

+ Pasos de tamaño constante. En este caso, el tamaño de paso determina la precisión de la solución.
+ Pasos de tamaño decreciente.
+ En cada paso, seleccionar el paso óptimo.

La última política es buena, pero demasiado cara.

## 1.3. De las derivadas al gradiente: minimización de funciones $n$-dimensionales.
Consideremos una función $n$-dimensional $f: \mathbf{R}^n \rightarrow \mathbf{R}$. 
Por ejemplo:

$$f(\mathbf{x}) = \suma_{n} x_n^2$$
Nuestro objetivo es encontrar el argumento $\mathbf{x}$ que minimice esta función.

El gradiente de $f$ es el vector cuyas componentes son las $n$ derivadas parciales de $f$. Se trata pues de una función vectorial.

El gradiente apunta en la dirección de la mayor tasa de incremento de la función.

$$\nabla {f} = (\frac{\parcial f}{\parcial x_1}, \dots, \frac{\parcial f}{\parcial x_n})$$

In [ ]:
def f(x):
    return sum(x_i**2 for x_i in x)

def fin_dif_partial_centered(x, 
                             f, 
                             i, 
                             h=1e-6):
    '''
    This method returns the partial derivative of the i-th 
    component of f at x
    by using the centered finite difference method
    '''
    w1 = [x_j + (h if j==i else 0) for j, x_j in enumerate(x)]
    w2 = [x_j - (h if j==i else 0) for j, x_j in enumerate(x)]
    return (f(w1) - f(w2))/(2*h)

def gradient_centered(x, 
                      f, 
                      h=1e-6):
    '''
    This method returns the gradient vector of f at x
    by using the centered finite difference method
    '''
    return[round(fin_dif_partial_centered(x,f,i,h), 10) for i,_ in enumerate(x)]


x = [1.0,1.0,1.0]

print('{:.6f}'.format(f(x)), gradient_centered(x,f))

La función que hemos evaluado, $f({\mathbf x}) = x_1^2+x_2^2+x_3^2$, es $3$ en $(1,1,1)$ y el vector gradiente en este punto es $(2,2,2)$. 

Entonces, podemos seguir estos pasos para maximizar (o minimizar) la función:

+ Partir de un vector aleatorio $\mathbf{x}$.
+ Calcular el vector gradiente.
+ Dar un pequeño paso en la dirección opuesta al vector gradiente.

> Es importante tener en cuenta que este cálculo del gradiente es muy costoso: si $\mathbf{x}$ tiene dimensión $n$, tenemos que evaluar $f$ en $2*n$ puntos.

# 2. Cómo aprender de los datos?

En general, *Aprender de los Datos* es una disciplina científica que se ocupa del diseño y desarrollo de algoritmos que permiten a las computadoras inferir, a partir de los datos, un modelo que permite una representación compacta de los datos crudos y/o buenas capacidades de generalización. En el primer caso, estamos hablando de aprendizaje no supervisado. En el segundo, de aprendizaje supervisado.

Hoy en día, esta es una tecnología importante porque permite a los sistemas computacionales mejorar su rendimiento con la experiencia acumulada a partir de los datos observados en escenarios del mundo real.

Consideremos el problema de aprendizaje supervisado desde un punto de vista de optimización. Al aprender un modelo a partir de datos, el escenario más común está compuesto por los siguientes elementos:

+ Un conjunto de datos $(\mathbf{x},y)$ de $n$ ejemplos. Por ejemplo, $(\mathbf{x},y)$ puede representar:

  + $\mathbf{x}$: el comportamiento de un jugador de videojuegos; $y$: pagos mensuales.
  + $\mathbf{x}$: datos de sensores del motor de tu coche; $y$: probabilidad de error del motor.
  + $\mathbf{x}$: datos financieros de un cliente de un banco; $y$: calificación del cliente.

  Si $y$ es un valor real, el problema que estamos intentando resolver se llama problema de *regresión*. Si $y$ es binario o categórico, se llama problema de *clasificación*.

+ Una función objetivo $f_{(\mathbf{x},y)}(\mathbf{w})$, que queremos minimizar, representando la discrepancia entre nuestros datos y el modelo que queremos ajustar.

+ Un modelo $M$ que está representado por un conjunto de parámetros $\mathbf{w}$.

+ El gradiente de la función objetivo, $\nabla {f_{(\mathbf{x},y)}(\mathbf{w})}$ con respecto a los parámetros del modelo.

En el caso de la regresión $f_{(\mathbf{x},y)}(\mathbf{w})$ representa los errores de un modelo de representación de datos $M$. Ajustar un modelo puede definirse como encontrar los parámetros óptimos $\mathbf{w}$ que minimicen la siguiente expresión:

$f_{(\mathbf{x},y)}(\mathbf{w}) = \frac{1}{n} \sum_{i} (y_i - M(\mathbf{x}_i,\mathbf{w}))^2$

Los problemas alternativos de regresión y clasificación se pueden definir considerando diferentes formulaciones para medir los errores de un modelo de representación de datos. Estas formulaciones se conocen como la *Función de Pérdida* del problema.


## 2.1. Square / Euclidean Loss

En problemas de regresión, la función de pérdida más común es la Square Loss:

$$ L(y, f(\mathbf{x})) = \frac{1}{n} \sum_i (y_i - f(\mathbf{x}_i))^2  $$

La función square loss se puede reescribir y utilizar para la clasificación:

$$ L(y, f(\mathbf{x})) = \frac{1}{n} \sum_i (1 - y_i f(\mathbf{x}_i))^2  $$


## 2.2. Hinge / Margin Loss (es decir, Support Vector Machine)

La función de hinge loss se define como:

$$ L(y, f(\mathbf{x})) = \frac{1}{n} \sum_i \mbox{max}(0, 1 - y_i f(\mathbf{x}_i))  $$

Hinge loss proporciona un límite superior convexo relativamente ajustado sobre la Pérdida 0–1.

## 2.3. Pérdida Logística (Regresión Logística)

Esta función muestra una tasa de convergencia similar a la función de hinge loss, y dado que es continua, se pueden utilizar métodos simples de gradient descend.

$$ L(y, f(\mathbf{x})) = \frac{1}{n} log(1 + exp(-y_i f(\mathbf{x}_i))) $$


## 2.4. Sigmoid Cross-Entropy Loss (clasificador Softmax)

La entropía cruzada es una función de pérdida muy utilizada para entrenar **problemas multiclase**. Nos centraremos en modelos que asumen que las clases son mutuamente excluyentes.

En este caso, nuestras etiquetas tienen esta forma $\mathbf{y}_i =(1.0,0.0,0.0)$. Si nuestro modelo predice una distribución diferente, digamos  $ f(\mathbf{x}_i)=(0.4,0.1,0.5)$, entonces nos gustaría ajustar los parámetros para que $f(\mathbf{x}_i)$ se acerque más a $\mathbf{y}_i$.

C.Shannon mostró que si quieres enviar una serie de mensajes compuestos por símbolos de un alfabeto con distribución $y$ ($y_j$ es la probabilidad del símbolo $j$-ésimo), entonces para usar el menor número de bits en promedio, deberías asignar $\log(\frac{1}{y_j})$ bits al símbolo $j$-ésimo.

El número óptimo de bits se conoce como **entropía**:

$$ H(\mathbf{y}) = \sum_j y_j \log\frac{1}{y_j} = - \sum_j y_j \log y_j$$

**Entropía cruzada** es el número de bits que necesitaremos si codificamos símbolos utilizando una distribución errónea $\hat y$:

$$ H(y, \hat y) =   - \sum_j y_j \log \hat y_j $$ 

En nuestro caso, la distribución real es $\mathbf{y}$ y la "errónea" es $f(\mathbf{x}_i)$. Por lo tanto, minimizar **la entropía cruzada** con respecto a nuestros parámetros del modelo resultará en el modelo que mejor aproxime nuestras etiquetas si se consideran como una distribución probabilística.

La entropía cruzada se utiliza en combinación con el clasificador **Softmax**. Para clasificar $\mathbf{x}_i$ podríamos tomar el índice correspondiente al valor máximo de $f(\mathbf{x}_i)$, pero Softmax ofrece una salida ligeramente más intuitiva (probabilidades de clase normalizadas) y también tiene una interpretación probabilística:

$$ P(\mathbf{y}_i = j \mid \mathbf{x_i}) = - log \left( \frac{e^{f_j(\mathbf{x_i})}}{\sum_k e^{f_k(\mathbf{x_i})} } \right) $$

donde $f_k$ es un clasificador lineal.

## 2.5. Descenso de gradiente por lotes (Batch Gradient Descend)

Podemos implementar **Gradient descend** de la siguiente manera (*descenso de gradiente por lotes*):

In [ ]:
import numpy as np
import random

# f = 2x
x = np.arange(10)
y = np.array([2*i for i in x])

# f_target = 1/n Sum (y - wx)**2
def target_f(x,y,w):
    return np.sum((y - x * w)**2.0) / x.size

# gradient_f = 2/n Sum 2wx**2 - 2xy
def gradient_f(x,y,w):
    return 2 * np.sum(2*w*(x**2) - 2*x*y) / x.size

def step(w,grad,alpha):
    return w - alpha * grad



def BGD(target_f, 
        gradient_f, 
        x, 
        y, 
        toler = 1e-6, 
        alpha=0.01):
    '''
    Batch gradient descend by using a given step
    '''
    w = random.random()
    val = target_f(x,y,w)
    i = 0
    while True:
        i += 1
        gradient = gradient_f(x,y,w)
        next_w = step(w, gradient, alpha)
        next_val = target_f(x,y,next_w)    
        if (abs(val - next_val) < toler):
            return w
        else:
            w, val = next_w, next_val
            
print('{:.6f}'.format(BGD(target_f, gradient_f, x, y)))

## 2.6. Stochastic Gradient Descend

La última función evalúa todo el conjunto de datos $(\mathbf{x}_i,y_i)$ en cada paso. 

Si el conjunto de datos es grande, esta estrategia es demasiado costosa. En este caso utilizaremos una estrategia llamada **SGD** (*Stochastic Gradient Descend*).

Al aprender de los datos, la función de coste es aditiva: se calcula sumando los errores de reconstrucción de las muestras. 

Entonces, podemos calcular la estimación del gradiente (y movernos hacia el mínimo) utilizando sólo **una muestra de datos** (o una pequeña muestra de datos).

Así, encontraremos el mínimo iterando esta estimación del gradiente sobre el conjunto de datos.

Una iteración completa sobre el conjunto de datos se denomina **epoch**. Durante un epoch, los datos deben utilizarse en un orden aleatorio.

Si aplicamos este método tenemos algunas garantías teóricas para encontrar un buen mínimo:
+ SGD utiliza esencialmente el gradiente inexacto por iteración. Puesto que no hay comida gratis, ¿cuál es el coste de utilizar el gradiente aproximado? La respuesta es que la velocidad de convergencia es más lenta que la del algoritmo de descenso del gradiente.
+ La convergencia de SGD se ha analizado utilizando las teorías de minimización convexa y de aproximación estocástica: converge casi con seguridad a un mínimo global cuando la función objetivo es convexa o pseudoconvexa, y en caso contrario converge casi con seguridad a un mínimo local.

In [ ]:
import numpy as np
x = np.arange(10)
y = np.array([2*i for i in x])
data = list(zip(x,y))

for (x_i,y_i) in data:
    print('{:3d} {:3d}'.format(x_i,y_i))
print("")

def in_random_order(data):
    '''
    Random data generator
    '''
    import random
    indexes = [i for i,_ in enumerate(data)]
    random.shuffle(indexes)
    for i in indexes:
        yield data[i]
        
import numpy as np
import random

def SGD(target_f, 
        gradient_f, 
        x, 
        y, 
        toler = 1e-6, 
        epochs=100, 
        alpha_0=0.01):
    '''
    Stochastic gradient descend with automatic step adaptation (by
    reducing the step to its 95% when there are iterations with no increase)
    '''
    data = list(zip(x,y))
    w = random.random()
    alpha = alpha_0
    min_w, min_val = float('inf'), float('inf')
    epoch = 0
    iteration_no_increase = 0
    while epoch < epochs and iteration_no_increase < 100:
        val = target_f(x, y, w)
        if min_val - val > toler:
            min_w, min_val = w, val
            alpha = alpha_0
            iteration_no_increase = 0
        else:
            iteration_no_increase += 1
            alpha *= 0.95
        for x_i, y_i in list(in_random_order(data)):
            gradient_i = gradient_f(x_i, y_i, w)
            w = w - (alpha *  gradient_i)
        epoch += 1
    return min_w
  
print('w: {:.6f}'.format(SGD(target_f, gradient_f, x, y)))

# 3. Mini-batch Gradient Descend (Descenso del gradiente por mini-lotes)

En código, el descenso del gradiente por lotes general se ve algo así:

```python
nb_epochs = 100
for i in range(nb_epochs):
    grad = evaluate_gradient(target_f, data, w)
    w = w - learning_rate * grad
```

Para un número predefinido de epochs, primero calculamos el vector gradiente de la función objetivo para todo el conjunto de datos con respecto a nuestro vector de parámetros. 

**Stochastic gradient descent** (SGD) en cambio, realiza una actualización de los parámetros para cada ejemplo de entrenamiento y etiqueta:

```python
nb_epochs = 100
for i in range(nb_epochs):
    np.random.shuffle(data)
    for sample in data:
        grad = evaluate_gradient(target_f, sample, w)
        w = w - learning_rate * grad
```

El **Mini-batch gradient descent** finalmente toma lo mejor de ambos mundos y realiza una actualización para cada minilote de $n$ ejemplos de entrenamiento:

```python
nb_epochs = 100
for i in range(nb_epochs):
  np.random.shuffle(data)
  for batch in get_batches(data, batch_size=50):
    grad = evaluate_gradient(target_f, batch, w)
    w = w - learning_rate * grad
```

El Mini-batch SGD tiene la ventaja de que trabaja con una estimación del gradiente ligeramente menos ruidosa. Sin embargo, a medida que aumenta el tamaño del minilote, disminuye el número de actualizaciones realizadas por cada cálculo efectuado (al final se vuelve muy ineficiente, como el Batch Gradient Descend). 

Existe un compromiso óptimo (en términos de eficiencia computacional) que puede variar en función de la distribución de los datos y de las particularidades de la clase de función considerada, así como de la forma en que se implementen los cálculos.

# 4. Diferenciación Automática

> El algoritmo de **backpropagation** (retropropagación) fue introducido originalmente en la década de 1970, pero su importancia no fue completamente apreciada hasta un famoso artículo de 1986 por David Rumelhart, Geoffrey Hinton y Ronald Williams. (Michael Nielsen en "Neural Networks and Deep Learning", http://neuralnetworksanddeeplearning.com/chap2.html).

> **backpropagation** es el algoritmo clave que hace que el entrenamiento de deep models sea computacionalmente viable. Para las redes neuronales modernas, puede hacer que el entrenamiento con gradient descend sea hasta diez millones de veces más rápido, en comparación con una naive implementation. Esa es la diferencia entre un modelo que tarda una semana en entrenarse y otro que tardaría 200,000 años. (Christopher Olah, 2016)

Hemos visto que, para optimizar nuestros modelos, necesitamos calcular la derivada de la función de pérdida respecto a todos los parámetros del modelo.

El cálculo de derivadas en modelos computacionales se aborda mediante cuatro métodos principales:

+ trabajando manualmente las derivadas y codificando el resultado (como en el artículo original que describe la retropropagación);
+ diferenciación numérica (usando aproximaciones de diferencia finita);
+ diferenciación simbólica (usando manipulación de expresiones en software, como Sympy);
+ y diferenciación automática (AD).

La **diferenciación automática** (AD) funciona aplicando sistemáticamente la **regla de la cadena** del cálculo diferencial a nivel del operador elemental.

Supongamos $ y = f(g(x)) $ nuestra función objetivo. En su forma básica, la regla de la cadena establece:

$$ \frac{\partial y}{\partial x} = \frac{\partial y}{\partial g} \frac{\partial g}{\partial x} $$

o, si hay más de una variable $g_i$ entre $y$ y $x$ (por ejemplo, si $f$ es una función bidimensional como $f(g_1(x), g_2(x))$), entonces:

$$ \frac{\partial y}{\partial x} = \sum_i \frac{\partial y}{\partial g_i} \frac{\partial g_i}{\partial x} $$

> Ver https://www.math.hmc.edu/calculus/tutorials/multichainrule/

Ahora, veamos cómo la AD permite la evaluación precisa de derivadas con machine precision, con solo un pequeño factor constante de sobrecarga.

En su descripción más básica, AD se basa en el hecho de que todos los cálculos numéricos
son en última instancia composiciones de un conjunto finito de operaciones elementales para las cuales se conocen las derivadas.

Por ejemplo, consideremos el cálculo de la derivada de esta función, que representa un modelo de red neuronal de 1 capa:

$$
    f(x) = \frac{1}{1 + e^{- ({w}^T \cdot  x + b)}} 
$$

Primero, escribamos cómo evaluar $f(x)$ a través de una secuencia de operaciones primitivas:


```python
x = ?
f1 = w * x
f2 = f1 + b
f3 = -f2
f4 = 2.718281828459 ** f3
f5 = 1.0 + f4
f = 1.0/f5
```

El signo de interrogación indica que $x$ es un valor que debe proporcionarse. 

Este *programa* puede calcular el valor de $x$ y también **poblar variables del programa**. 

Podemos evaluar $\frac{\parcial f}{\parcial x}$ en algún $x$ utilizando la regla de la cadena. Esto se llama *forward-mode differentiation*. 

En nuestro caso:

In [ ]:
def f(x,w,b):
    f1 = w * x
    f2 = f1 + b
    f3 = -f2
    f4 = 2.718281828459 ** f3
    f5 = 1.0 + f4
    return 1.0/f5

def dfdx_forward(x, w, b):
    f1 = w * x
    p1 = w                            # p1 = df1/dx
    f2 = f1 + b
    p2 = p1 * 1.0                     # p2 = p1 * df2/df1 
    f3 = -f2
    p3 = p2 * -1.0                    # p3 = p2 * df3/df2
    f4 = 2.718281828459 ** f3
    p4 = p3 * 2.718281828459 ** f3    # p4 = p3 * df4/df3
    f5 = 1.0 + f4
    p5 = p4 * 1.0                     # p5 = p4 * df5/df4
    f = 1.0/f5
    df = p5 * -1.0 / f5 ** 2.0        # df/dx = p5 * df/df5
    return f, df

der = (f(3+0.00001, 2, 1) - f(3, 2, 1))/0.00001

print("Value of the function at (3, 2, 1): ",f(3, 2, 1))
print("df/dx Derivative (fin diff) at (3, 2, 1): ",der)
print("df/dx Derivative (aut diff) at (3, 2, 1): ",dfdx_forward(3, 2, 1)[1])

Es interesante observar que este *programa* puede derivarse automáticamente si tenemos acceso a **subrutinas que implementan las derivadas de funciones primitivas** (como $\exp{(x)}$ o $1/x$) y todas las variables intermedias se calculan en el orden correcto. 

También es interesante señalar que AD permite la evaluación exacta de las derivadas a **machine precision**, con sólo un pequeño factor constante de sobrecarga.

Forward differentiation es eficiente para funciones $f : \mathbb{R}^n \rightarrow \mathbb{R}^m$ con $n << m$ (sólo se necesitan $O(n)$ barridos). 

Para los casos $n >> m$ se necesita una técnica diferente. Para ello, reescribiremos la regla de la cadena como:

$$
\frac{parcial f}{parcial x} = \frac{parcial g}{parcial x} \frac{parcial f}{parcial g}
$$

para propagar derivadas hacia atrás desde una salida dada. Esto se denomina *diferenciación en modo inverso*. El paso inverso comienza en el final (es decir, $\frac{\partial f}{\partial f} = 1$) y se propaga hacia atrás a todas las dependencias.


In [ ]:
def dfdx_backward(x, w, b):
    import numpy as np
    f1 = w * x
    f2 = f1 + b
    f3 = -f2
    f4 = 2.718281828459 ** f3
    f5 = 1.0 + f4
    f = 1.0/f5
    
    pf = 1.0                           # pf = df/df
    p5 = 1.0 * -1.0 / (f5 * f5) * pf   # p5 = pf * df/df5 
    p4 = p5 * 1.0                      # p4 = p5 * df5/df4
    p3 = p4 * np.log(2.718281828459) \
          * 2.718281828459 ** f3       # p3 = p4 * df4/df3
    p2 = p3 * -1.0                     # p2 = p3 * df3/df2
    p1 = p2 * 1.0                      # p1 = p2 * df2/df1
    dfx = p1 * w                       # dfx = p1 * df1/dx 
    return f, dfx

print("df/dx Derivative (aut diff) at (3, 2, 1): ",
      dfdx_backward(3, 2, 1)[1])

Cualquier función compleja que pueda descomponerse en un conjunto de funciones elementales puede derivarse de forma automática, con machine precision, mediante este algoritmo.

**¡Ya no necesitamos codificar derivadas complejas para aplicar SGD!**